# OpenLDAP

## dockerhostの設定

In [ ]:
%%bash
### ldapクライアントのインストール(CentOS7)
sudo yum install -y openldap openldap-clients

## コンテナの作成

In [1]:
%%bash
### 変数の設定
container="openldap_centos7"
docker_compose_file="docker-compose_${container}.yml"
ldap_port="389"
ssh_port="2223"

### イメージ、コンテナが無い状態の確認
docker images
docker ps -a

### docker-composeディレクトリに移動
#work_dir="/home/jsd/work/build/docker-compose"
work_dir="${PYENV_DIR}/work/build/docker-compose"
cd ${work_dir}

### docker-composeファイルの作成
cat <<- EOS > ${docker_compose_file}
${container}:
   image: ftakao2007/openldap_centos7:latest
   container_name: ${container}
   privileged: true
   tty: true
   ports:
     - "${ldap_port}:389"
     - "${ssh_port}:22"
   #command: sh -c '/sbin/init'
   command: sh -c 'exec /usr/sbin/init && systemctl restart sshd.service && systemctl restart slapd.service'
EOS

### コンテナの作成
docker-compose -f ${docker_compose_file} up -d
#sleep 80

### イメージ、コンテナの確認
docker images
docker ps -a

REPOSITORY                    TAG                      IMAGE ID            CREATED             SIZE
ftakao2007/mail_centos7       virtual_for_jorurimail   35e0eebeeb79        17 hours ago        657MB
ftakao2007/mail_centos7       latest                   0733ee002c86        18 hours ago        656MB
ftakao2007/openldap_centos7   latest                   572b38993fa4        2 days ago          596MB
ftakao2007/mysql5.6_centos7   latest                   09a773ad8e34        4 days ago          2.16GB
ftakao2007/centos7            latest                   0fede72fa1fe        4 days ago          488MB
CONTAINER ID        IMAGE                                COMMAND                  CREATED             STATUS              PORTS                                          NAMES
4c0ae849ad6a        ftakao2007/mysql5.6_centos7:latest   "sh -c 'exec /usr/..."   2 minutes ago       Up 2 minutes        0.0.0.0:3306->3306/tcp, 0.0.0.0:2222->22/tcp   mysql5.6_centos7
REPOSITORY                    TAG

Creating openldap_centos7


In [3]:
%%bash
### ldap接続確認
dn="dc=dockernet,dc=co,dc=jp"
search="(&(objectClass=person)(mail=joruri_test11@dockernet.co.jp))"
ldapsearch -x -LLL -b "${dn}" -h 127.0.0.1 "${search}"

dn:: dWlkPTAwNTAwMTEsb3U9MDAxMDAy56eY5pu45bqD5aCx6KqyLG91PTAwMeS8geeUu+mDqCxkY
 z1kb2NrZXJuZXQsZGM9Y28sZGM9anA=
o:: MDAx5LyB55S76YOo
ou:: MDAxMDAy56eY5pu45bqD5aCx6Kqy
title:: 5LiA6Iis56S+5ZOh
departmentNumber: 001003001011
givenName: joruri_test
givenName;lang-en: joruri_test
sn:: 44K444On44O844Or44Oq
sn;lang-en: joruri_test
cn:: 44K444On44O844Or44Oq44OG44K544OIMQ==
cn;lang-en: joruri
uidNumber: 10000001
gidNumber: 20000001
homeDirectory: /home/joruri_test11
objectClass: inetOrgPerson
objectClass: organizationalPerson
objectClass: person
objectClass: posixAccount
objectClass: shadowAccount
preferredLanguage: ja
st:: am9ydXJp5biC
mail: joruri_test11@dockernet.co.jp
userPassword:: e0NSWVBUfXppNXNhZnAvSHllWGs=
uid: 0050011



## VIPの付与

In [4]:
%%bash
### ネットワーク情報の確認
container="openldap_centos7"
docker exec ${container} ifconfig -a 

eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.17.0.3  netmask 255.255.0.0  broadcast 0.0.0.0
        ether 02:42:ac:11:00:03  txqueuelen 0  (Ethernet)
        RX packets 19  bytes 1501 (1.4 KiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 8  bytes 1238 (1.2 KiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        loop  txqueuelen 1  (Local Loopback)
        RX packets 0  bytes 0 (0.0 B)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 0  bytes 0 (0.0 B)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0



In [5]:
%%bash
### VIPを付与する
container="openldap_centos7"
ip="172.17.33.12"
netmask="255.255.0.0"
device="eth0"
docker exec ${container} ifconfig ${device}:1 ${ip} netmask ${netmask}

### ネットワーク確認
docker exec ${container} ifconfig -a

### LDAP接続確認
dn="dc=dockernet,dc=co,dc=jp"
search="(&(objectClass=person)(mail=joruri_test11@dockernet.co.jp))"
ldapsearch -x -LLL -b "${dn}" -h ${ip} "${search}" | head

eth0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.17.0.3  netmask 255.255.0.0  broadcast 0.0.0.0
        ether 02:42:ac:11:00:03  txqueuelen 0  (Ethernet)
        RX packets 19  bytes 1501 (1.4 KiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 8  bytes 1238 (1.2 KiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

eth0:1: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 172.17.33.12  netmask 255.255.0.0  broadcast 172.17.255.255
        ether 02:42:ac:11:00:03  txqueuelen 0  (Ethernet)

lo: flags=73<UP,LOOPBACK,RUNNING>  mtu 65536
        inet 127.0.0.1  netmask 255.0.0.0
        loop  txqueuelen 1  (Local Loopback)
        RX packets 0  bytes 0 (0.0 B)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 0  bytes 0 (0.0 B)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0

dn:: dWlkPTAwNTAwMTEsb3U9MDAxMDAy56eY5pu45bqD5aCx6KqyLG91PTAwMeS8geeUu+mDqCxkY
 z1kb2NrZXJu